# Analyse dynamique des déterminants des prix immobiliers et de l'impact des taux d'intérêt sur le marché du logement

Ce projet python vise à étudier l'influence des taux d'intérêts sur les prix immobilier au niveau local en fonction de divers charactéristiques. L'objectif principal est de déterminer si l'impact des taux d'interet peut-être observé de façon homogène sur le territoire français. La question de l'effet des taux sur le marché immobilier est traité de façons assez large dans la littérature, souvent à des niveaux assez agrégé comme un pays (voir un working paper de la BCE _House prices and ultra-low interest rates: exploring the non-linear nexus_, Dieckelmann et al). Ce projet a donc la particularité de travailler à un niveau bien plus micro, l'échelon local. 

Notre analyse est limité à la période de disponibilité des données "Demandes de Valeurs Foncières" (2014-2023) qui listent toutes les transactions immobilières réalisées sur le territoire français, et nous permettent de construire des bases locales des marchés immobiliers. 

Ce notebook peut se décomposer en 3 grandes parties :
__________________________________________________
Partie I - Préparation de la DVF
 1. Récupération des données 
 2. Agrégation en utilisant des tables de passage
___________________________________________________
 3. Visualisation et étude des données de la DVF
___________________________________________________

Partie II - Variables de controle
 1. Récupération des données
 2. Création d'une table variable de contrôle 
 3. Visualisation des données de controle
___________________________________________________
Partie III - Analyse, modélisation et économétrie
 1. Test de Granger
 2. Corrélations 
 3. Modèles communaux
 4. Modèle avec prix dé-tendanciés
 5. Modèle ZE
 6. Modèle EPCI
____________________________________________________
Pour faciliter le contrôle de version, notre travail a été initialement réalisé sur 4 scripts indépendants ce qui représente un avantage innatendu puisque le code présent dans ce notebook peut être utilisé de façon indépendante entre 4 sections indiquées plus haut par des lignes horizontales, seule les importations des dependencies et la selection du path doivent toujours être éxécuté. 

## Les importations 

In [ ]:
import pandas as pd
import numpy as np
import pyarrow.feather as feather
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.stattools import grangercausalitytests
from linearmodels import PanelOLS, RandomEffects
from statsmodels.stats.diagnostic import het_breuschpagan
from linearmodels.panel import compare
from statsmodels.tsa.stattools import adfuller
warnings.filterwarnings('ignore')

In [ ]:
# Change path as needed
path = "C:/Users/patry/Documents/GitHub/interestnpy"

## Partie I - Préparation de la DVF

### I.1. Récupération de données 

#### Création d'une table de passage

La géographie des différents agrégats locaux en France a changé entre 2014 et 2023, ce qui nécessite donc de construire une table de passage. Pour cela nous utilisons une table constante des code communes de l'INSEE que nous adaptons à trois niveaux de travail, commune, zone d'emploi et intercomunalité.

In [ ]:
#IMPORTS AND FILTER INSEE TRANSLATION TABLE
passage1423_df = pd.read_excel(path+"/data/external/GEOCOM/table_passage_annuelle_2023.xlsx", 
                               sheet_name="COM", skiprows=5)
passage1423_df = passage1423_df.filter(regex='^CODGEO').iloc[:, :10]
passage1423_df = passage1423_df[passage1423_df['CODGEO_2023'].notna()]

In [ ]:
#IMPORTS 2023 EPCI DEFINITIONS
fcom_df = pd.read_excel(path+"/data/external/GEOCOM/ZE2020_au_01-01-2023.xlsx", 
                            sheet_name="Composition_communale", skiprows=5)

epc_com = pd.read_excel(path+"/data/external/GEOCOM/Intercommunalite_Metropole_au_01-01-2023.xlsx", 
                            sheet_name="Composition_communale", skiprows=5)

fcom_df = pd.merge(fcom_df, epc_com, how='left', on=['CODGEO','LIBGEO', 'DEP', 'REG'])

Le niveau intercommunalité des 3 plus grandes zones métropolitaines maintient les métropoles indépendantes.

In [ ]:
#CREATES COM BASED ON 2023 TOWN CODES AND ZE BASED ON EPCI
fcom_df['COM'] = fcom_df['CODGEO']
fcom_df['LIB_COM'] = fcom_df['LIBGEO']
fcom_df['ZE'] = fcom_df['ZE2020']
fcom_df['LIB_ZE'] = fcom_df['LIBZE2020']
fcom_df['EPCI'] = fcom_df['ZE2020']
fcom_df['LIB_ZE'] = fcom_df['LIBZE2020']
fcom_df['EPCI'] = fcom_df.apply(
    lambda row: row['CODGEO'] if any(x in row['LIBEPCI'] for x in ["Métropole du Grand Paris", "Métropole de Lyon", "Métropole d'Aix-Marseille-Provence"]) else row['EPCI'],
    axis=1
)
fcom_df['LIB_EPCI'] = fcom_df.apply(
    lambda row: row['LIBGEO'] if any(x in row['LIBEPCI'] for x in ["Métropole du Grand Paris", "Métropole de Lyon", "Métropole d'Aix-Marseille-Provence"]) else row['LIBEPCI'],
    axis=1
)
fcom_df['LIB_EPCI'] = fcom_df.apply(
    lambda row: "Iles-FR" if row['EPCI'] == "ZZZZZZZZZ" else row['LIB_EPCI'],
    axis=1
)
fcom_df.rename(columns={'CODGEO': 'CODGEO_2023'}, inplace=True)
fcom_df = fcom_df[['CODGEO_2023', 'COM', 'LIB_COM', 'ZE', 'LIB_ZE', 'EPCI', 'LIB_EPCI']]

In [ ]:
#MERGES WITH INSEE TABLE TO CREATE THE TRANSLATION TABLE 
ident1423_df = pd.merge(passage1423_df, fcom_df, how='left', on='CODGEO_2023')

In [ ]:
#FIXES PROBLEMS RELATED TO CITIES WITH "ARRONDISSEMENTS"
conditions_arr = [
    ident1423_df['CODGEO_2023'].str.startswith("75"),
    ident1423_df['CODGEO_2023'].str.startswith("132"),
    ident1423_df['CODGEO_2023'].str.startswith("6938")
]

ze_arr = [1109, 9312, 8421]
lib_arr = ["Paris", "Marseille", "Lyon"]

ident1423_df['COM'] = ident1423_df.apply(
    lambda row: row['CODGEO_2023'] if pd.isna(row['COM']) else row['COM'],
    axis=1
)

ident1423_df['EPCI'] = ident1423_df.apply(
    lambda row: row['CODGEO_2023'] if pd.isna(row['EPCI']) else row['EPCI'],
    axis=1
)
ident1423_df['EPCI'] = ident1423_df['EPCI'].astype(str)

ident1423_df['ZE'] = np.select(
  conditions_arr, 
  ze_arr, 
  default=ident1423_df['ZE'])

In [ ]:
def create_lib_ident(row, col_name):
    if pd.isna(row[col_name]):
        if row['COM'][:2] == "75":
            return f"Paris {row['COM'][3:5]}"
        elif row['COM'][:2] == "69":
            return f"Lyon {int(row['CODGEO_2019']) - 80}"
        elif row['COM'][:2] == "13":
            return f"Marseille {row['COM'][3:5]}"
    else:
        return row[col_name]

In [ ]:
ident1423_df['LIB_COM'] = ident1423_df.apply(create_lib_ident, col_name='LIB_COM', axis=1)

ident1423_df['LIB_EPCI'] = ident1423_df.apply(
    lambda row: row['LIB_COM'] if pd.isna(row['LIB_EPCI']) else row['LIB_EPCI'],
    axis=1
) 

ident1423_df['LIB_ZE'] = np.select(
  conditions_arr, 
  lib_arr, 
  default=ident1423_df['LIB_ZE'])

In [ ]:
#REORDERS AND EXPORTS THE TRANSLATION TABLE
reord_cols = ['COM', 'ZE', 'LIB_COM', 'LIB_ZE', 'EPCI', 'LIB_EPCI']
ident1423_df = ident1423_df[reord_cols + [col for col in ident1423_df.columns if col not in reord_cols]]
ident1423_df.to_feather(path+"/data/interim/tble_de_passage_py.feather")

In [ ]:
ident1423_df.head()

#### Prix des terrains par département

Les données du Ministère de la Transition écologique sur les prix des terrains par région de 2008 à 2022 permettent à l'étape d'agrégation de soustraire les prix des terrains des montants des transactions dans les données DVF.

In [ ]:
#LOADS CONVERSION TABLE REG - DEP
depreg_df = pd.read_excel(path+"/data/external/GEOCOM/table-appartenance-geo-communes-19.xls", 
                          sheet_name="COM", skiprows=5)[['DEP', 'REG']].drop_duplicates()
depreg_df['REG'] = depreg_df['REG'].astype(str)

#LOADS REGIONAL PRICES FOR LAND
ter_df = pd.read_csv(path+"/data/external/TER/1-Terrains-achetes-nombre-surface-et-prix-moyen-par-region.2022-01.csv", 
                     delimiter=";", skiprows=1, escapechar='\\', skipinitialspace=True)

In [ ]:
#CREATING ADDITIONAL CONSTANT VALUES FOR 2023
additional_rows_2023 = ter_df[ter_df['ANNEE'] == 2022].copy()
additional_rows_2023['ANNEE'] = 2023
ter_df = pd.concat([ter_df, additional_rows_2023]).sort_values('ANNEE')
ter_df = ter_df[ter_df['ANNEE'] > 2013]

In [ ]:
#MERGES TO GO FROM REGIONAL LEVEL TO DEPARTMENT LEVEL AND EXPORTS
terdep_df = pd.merge(depreg_df, ter_df, left_on='REG', right_on='ZONE_CODE', how='left')
terdep_df = terdep_df.dropna().reset_index(drop=True)[['ANNEE', 'DEP', 'PTM2_MED']]
terdep_df.to_feather(path+"/data/interim/terrains_py.feather")

In [ ]:
terdep_df.head()

#### Inflation - Taux d'intérêt

Dans le cadre d'une analyse économique, il est préférable de travailler en euros réels, c'est-à-dire corrigé de l'inflation. Nous construisons donc une table de l'inflation cumulée depuis 2014 jusqu'à la fin du premier semestre 2023 à partir de données de l'INSEE. De l'inflation cumulée par année est extrapolée des données trimestrielles. 

In [ ]:
#INFLATION TABLE
inflation = pd.read_excel(path+"/data/external/INFLATION/econ-gen-taux-inflation.xlsx", 
                          sheet_name="Données", skiprows=3, nrows=9)
inflation.rename(columns={"Année": "AN", "Taux d'inflation": "INFLATION"}, inplace=True)
inflation['AN'] = inflation['AN'].astype(str)
inflation = inflation._append({'AN': str(2023), 'INFLATION': 2.3}, ignore_index=True)
#Why 2.3? first 3 months inflation
#either way it does not matter as we will work in basis 2023
inflation.sort_values(by="AN", inplace=True)
inflation['Date'] = inflation['AN']+"0101"
inflation.loc[len(inflation)] = ["2023",0, "20230401"]
inflation['TOT'] = (1 + inflation['INFLATION'].shift(1) / 100).cumprod()
inflation['TOT'] = inflation['TOT'].fillna(1)

Les taux d'intérêts utilisés dans le cadre de ce projet ne sont pas les taux directeurs de la BCE comme cela peut-être observé dans la littérature, mais les taux de crédit immobiliers moyen observé par la Banque de France. Le projet ne cherche donc pas à étudier le délai de transmission de la politique monétaire de la BCE vers le marché immobilier français, mais directement de la transmission du taux d'intérêt effectivement pratiqué.

In [ ]:
# INTEREST RATES TABLE
interest_rates = pd.read_excel(path+"/data/external/INFLATION/series_panorama_202309.xlsx", 
                               sheet_name="G3", 
                               skiprows=6, 
                               names=["Date", "ir", "ir10_avg", "ir20_avg"])

interest_rates = interest_rates[interest_rates['Date'] > '2013-12-31']

interest_rates['quarter'] = interest_rates['Date'].dt.to_period('Q').astype(str)
interest_rates = interest_rates.groupby('quarter').agg({'ir': 'mean'}).reset_index()

In [ ]:
def get_month_from_quarter(quarter):
    quarter_num = quarter.split("Q")[1]
    return {'1': '01', '2': '04', '3': '07', '4': '10'}.get(quarter_num, None)

In [ ]:
interest_rates['themonth'] = interest_rates['quarter'].apply(get_month_from_quarter)
interest_rates['Date'] = (interest_rates['quarter'].str[:4] + interest_rates['themonth'] + "01")
interest_rates = interest_rates[['Date', 'ir']]

In [ ]:
# Combine inflation and interest rates
irflation = pd.merge(interest_rates, inflation, on='Date', how='left')
irflation = irflation.interpolate(method="slinear", fill_value="extrapolate", limit_direction="both")
irflation['BASE14'] = irflation['TOT'] / irflation['TOT'].iloc[0]
irflation['BASE23'] = irflation['BASE14'] / irflation['BASE14'].iloc[37]
irflation.to_feather(path+"/data/interim/irflation.feather")

In [ ]:
irflation.head()

### I-2. Agrégation des données de la DVF

Dans cette partie, on agrège les données DVF trimestriellement en utilisant une fonction personnalisée à 3 niveaux distincts : commune, EPCI et zone d'emploi. Les données ont été filtrées, seul les données métropolitaines sont conservées et les transactions concernant un local commercial ou une parcelle ont été supprimées. Pour chaque niveau d'agrégation, 3 variables d'intérêt ont été gardées : le nombre de transactions, le prix médian et la part des maisons dans le total des transactions. Cette proportion est utilisée comme variable de contrôle pour les différences structurelles du marché de l'immobilier des différentes communes, car les maisons sont souvent moins chères au mètre carré que les appartements.

In [ ]:
terdep_df = pd.read_feather(path+"/data/interim/terrains_py.feather")
ident_df = pd.read_feather(path+"/data/interim/tble_de_passage_py.feather").drop_duplicates()

In [ ]:
def aggreg_fun(data, ID, LIB_ID):
    quantiles = data.groupby([ID, LIB_ID, 'Date', 'Type local'])['prixM2'].quantile([0.05, 0.95]).unstack()
    quantiles.columns = ['quantile_05', 'quantile_95']
    data = data.merge(quantiles, left_on=[ID, LIB_ID, 'Date', 'Type local'], right_index=True)
    filtered_data = data[(data['prixM2'] <= data['quantile_95']) & (data['prixM2'] >= data['quantile_05'])]
    print("Data filtered!")
    data_agreg = filtered_data.groupby([ID, LIB_ID,'Date']).agg(
     n_transactions=('prixM2', 'size'),
     prop_maison=('Type local', lambda x: np.mean(x == 'Maison')),
    prixM2=('prixM2', 'median')
    ).reset_index()
    return data_agreg

Remarque : Pour réduire au maximum le poids des valeurs extrêmes ont été supprimés les 5% des prix au m2 les plus faibles et les 5% les plus élevés (par exemple, les appartements dans Paris 6e vendu à 1500€/m2 n'ont pas été pris en compte). 

In [ ]:
def immo_prices(year):
    # Read data
    df = pd.read_csv(path+f"/data/external/DFV/valeursfoncieres-{year}.txt", delimiter="|", dtype=str)
    df = df.dropna(axis=1, how='all')  
    # Filter and mutate
    df = df[df['Nature mutation'] == "Vente"]
    df = df[df['Code type local'].isin(['1', '2'])]
    df = df[~df['Code departement'].isin(['971', '972', '973', '974'])]
    df['Code commune'] = df['Code commune'].str.pad(width=3, side='left', fillchar='0')
    df['depcom'] = df['Code departement'] + df['Code commune']
    df['Valeur fonciere'] = df['Valeur fonciere'].str.replace(",", ".").astype(float)
    print("Imported!")
    # Quarter and date manipulation
    df['quarter_num'] = df['Date mutation'].str[3:5].astype(int)
    df['qmonth'] = np.select([df['quarter_num'].isin(range(1, 4)), df['quarter_num'].isin(range(4, 7)), df['quarter_num'].isin(range(7, 10)), df['quarter_num'].isin(range(10, 13))], ["01", "04", "07", "10"], default=np.nan)
    df['Date'] = (str(year) + df['qmonth'] + "01")
    print("Date computed!")
    # Merging with ident_year_df
    ident_year_df = ident_df[['COM', 'EPCI', 'ZE', 'LIB_COM', 'LIB_EPCI', 'LIB_ZE', f'CODGEO_{year}']].drop_duplicates()
    df = df.merge(ident_year_df, left_on='depcom', right_on=f'CODGEO_{year}', how='left')
    # Merging with terdep_df
    df = df.merge(terdep_df[terdep_df['ANNEE'] == int(year)], left_on='Code departement', right_on='DEP', how='left')
    # Calculating prixM2
    df['Surface terrain'] = df['Surface terrain'].replace(np.nan, 0).astype(float)
    df['prixM2'] = (df['Valeur fonciere'] - df['Surface terrain'] * df['PTM2_MED']) / df['Surface reelle bati'].astype(float)
    df = df[(df['prixM2'] > 10) & (df['prixM2'] < 100000)].drop_duplicates()
    print("Price computed!")
    df = df.drop_duplicates(subset=['Date mutation', 'No voie', 'Valeur fonciere', 'Surface terrain', 'LIB_COM'])
    df4_epci = aggreg_fun(df,"EPCI","LIB_EPCI")
    print("EPCI done!")
    df4_com = aggreg_fun(df,"COM", "LIB_COM")
    print("Town done!")
    df4_ze = aggreg_fun(df,"ZE","LIB_ZE")
    print("ZE done!")
    df4_full = df.groupby('Date', as_index=False).agg(prixM2=('prixM2', 'mean'))


    return [df4_com, df4_epci, df4_ze, df4_full]

**Attention! L'agrégation ci-dessous prends entre 3 et 10 min selon le processeur**

In [ ]:
years = ["2023", "2022", "2021", "2020", "2019", "2018", "2017", "2016", "2015", "2014"]
results = []

for i in years:
  print("Computing "+i)  
  results.append(immo_prices(i))

In [ ]:
# Merging results
immo1423_com = pd.concat([result[0] for result in results])
immo1423_epci = pd.concat([result[1] for result in results])
immo1423_ze = pd.concat([result[2] for result in results])
immo1423_full = pd.concat([result[3] for result in results])

In [ ]:
# Writing output
feather.write_feather(immo1423_com, path+"/data/interim/immo_panel_com_py.feather")
feather.write_feather(immo1423_epci, path+"/data/interim/immo_panel_epci_py.feather")
feather.write_feather(immo1423_ze, path+"/data/interim/immo_panel_ze_py.feather")
feather.write_feather(immo1423_full, path+"/data/interim/immo_panel_full_py.feather")

### I-3. Visualisation de données 

En utilisant des libraries comme geopandas et des shapefiles de l'INSEE nous pouvons visualiser certaines de nos données géographiques issues de la DVF. Seul le niveau zone d'emploi est considéré les autres niveaux étant trop petits pour être bien lisible sur une carte.

In [ ]:
# Reading the data
ze_shp = gpd.read_file(path+"/data/external/ze2020_2023/ze2020_2023.shp")
immo_panel = pd.read_feather(path+"/data/interim/immo_panel_ze_py.feather")
immo_panel_com = pd.read_feather(path+"/data/interim/immo_panel_com_py.feather")
immo_panel_full = pd.read_feather(path+"/data/interim/immo_panel_full_py.feather")
irflation = pd.read_feather(path+"/data/interim/irflation.feather")
interest_rates = pd.read_feather(path+"/data/interim/interest_rates.feather")

In [ ]:
# Data manipulation
immo_panel = immo_panel[(immo_panel['Date'] == '20230101') | 
                           (immo_panel['Date'] == '20190101') | 
                           (immo_panel['Date'].isna())]
immo_panel.sort_values(by=['ZE', 'Date'], inplace=True)
immo_panel['dprix'] = immo_panel.groupby('ZE')['prixM2'].transform(lambda x: x - x.shift())

ze_shp['ZE'] = ze_shp['ze2020'].astype(float)
full_map = ze_shp.merge(immo_panel, on="ZE", how="left")

In [ ]:
# Plotting
def plot_map(data, variable, title, palette):
    fig, ax = plt.subplots(figsize=(10, 6))
    data.plot(column=variable, ax=ax, legend=True, cmap=palette,
    missing_kwds={'color': 'lightgrey', 'hatch': '///'})
    plt.title(title)
    plt.xlim(-4.5, 9.5)
    plt.ylim(41.5, 51)
    ax.set_axis_off()
    plt.show()

In [ ]:
plot_map(full_map[(full_map['Date'] == '20190101')|(full_map['Date'].isna())], 
         'prixM2', "Prix immobilier par zone d'emploi, premier trimestre 2019", 
         "YlGnBu")

On observe une très forte hétérogénéité des prix de l'immobillier en France. Les prix sont très élevées dans la zone d'emploi de Paris et plus largement dans l'Ouest de l'Ile-de-France, ainsi que dans la Côte d'Azur. On remarque également des prix élevés dans les zones d'emploi proches de la frontière Suisse. Cela est probablement dû à la présence de travailleurs frontaliers. Au contraire, les prix immobilier sont faibles dans les zones rurales, en particulier dans le centre de la France.

Remarque: La DVF ne comprends pas de données pour une partie de l'Alsace et la Lorraine.


In [ ]:
plot_map(full_map[(full_map['Date'] == '20230101')|(full_map['Date'].isna())], 
         'dprix', "Prix immobilier par zone d'emploi, différence 2023T1-2019T1", 
         "RdYlGn_r")

Les prix réels de l'immobilier sont stables dans l'essentiel de la France. Cependant, certains territoires ont vu de fortes augmentations. Toutes les zones d'emplois situées sur la côte atlantique ont subi des augmentations importantes tout comme celles sur la côte de la méditerannées de la région PACA. On remarque aussi une forte augmentation dans les zones d'emplois proches de la frontière Suisse. L'augmentation des prix dans la zone d'emploi de Paris entre 2019 et 2023 a été d'environ 1000€m2. Une des plus forte hausse a été dans la zone de Sainte Maxime, avec plus de 1500€m2. 

In [ ]:
# Plotting time series
def plot_time_series(data, x, y, title, hue=None):
    sns.set_theme(style="whitegrid")
    plt.figure(figsize=(10, 6))
    sns.lineplot(data=data, x=x, y=y, hue=hue)
    plt.title(title)
    plt.show()

In [ ]:
plot_time_series(irflation, "AN", "INFLATION", "Inflation entre 2014 et 2023, en pourcents")

On observe qu'entre 2014 et 2016, la France a connu une inflation relativement basse. Puis, de 2016 à 2018, l'inflation a légèrement augmenté pour atteindre des niveaux plus proches de l'objectif de la Banque centrale européenne (BCE) qui vise une inflation inférieure mais proche de 2%. Ensuite, la crise du Covid en 2020 a de nouveau fait baisser l'inflation. En effet, les mesures de confinement ont contribué à une baisse de la demande et à une pression à la baisse des prix. Mais en 2022, l'inflation a fortement augmenté pour atteindre les 5%, cela est lié notamment à l'invasion de l'Ukraine par la Russie. L'inflation représenté sur ce graphique en 2023 concerne uniquement le premier trimestre. 

Ainsi, l'inflation impacte nos données principalement à partir de 2021.

In [ ]:
interest_rates['Date'] = pd.to_datetime(interest_rates['Date'], format='%Y%m%d')
plot_time_series(interest_rates, "Date", "ir", "Taux d'interet moyen entre 2014 et 2023, en pourcentage")

Le taux d'interet moyen pratiqué sur les crédits immobiliers a varié de façon assez significative sur la période étudié, ce qui devrait nous permettre de plus facilement identifier son effet sur les marchés locaux.

In [ ]:
immo_panel_com['DateF'] = pd.to_datetime(immo_panel_com['Date'], format='%Y%m%d')
immo_panel_com_grouped = immo_panel_com.groupby('DateF').agg({'n_transactions': 'sum'}).reset_index()
plot_time_series(immo_panel_com_grouped, 'DateF', 'n_transactions', 
                 "Évolution du nombre de transactions, France DVF, 2014T1-2023T2")

Le nombre trimestriel de transacitons augmente d'années en années jusqu'au millieu de l'année 2021, qui marque le début d'une baisse prononcé des transactions. Il semble y avoir une corrélation négative entre le taux d'interet et le nombre de transactions.

In [ ]:
np.corrcoef(immo_panel_com_grouped["n_transactions"], interest_rates[interest_rates["ir"]!=3.25]["ir"])

Cette corrélation est donc d'environ -0.67.

In [ ]:
immo_panel_full['DateF'] = pd.to_datetime(immo_panel_full['Date'], format='%Y%m%d')
plot_time_series(immo_panel_full, 'DateF', 'prixM2', 
                 "Évolution des prix immonilier, France DVF, 2014T1-2023T2")

In [ ]:
np.corrcoef(immo_panel_full["prixM2"], interest_rates[interest_rates["ir"]!=3.25]["ir"])

Étonnement, il semble y avoir une corrélations positive entre le taux d'interet et les prix immobliers en France sur la période. Cependant, il s'agit très probablement d'une corrélation falacieuse liée à la non stationarité des prix immobiliers. 

In [ ]:
# Evolution des prix parisiens ---
paris_data = immo_panel_com[immo_panel_com['LIB_COM'].str.startswith('Paris ')]
paris_data = paris_data.sort_values(by='DateF')
window_size = 5  
paris_data['Moving_Average'] = paris_data.groupby('LIB_COM')['prixM2'].transform(lambda x: x.rolling(window=window_size).mean())
plt.figure(figsize=(12, 8))
palette = sns.color_palette("husl", n_colors=len(paris_data['LIB_COM'].unique()))
sns.lineplot(data=paris_data, x='DateF', y='prixM2', hue='LIB_COM',palette=palette,alpha=0.3)
sns.lineplot(data=paris_data, x='DateF', y='Moving_Average', hue='LIB_COM',palette=palette, alpha=0.8, legend=None)
plt.title('Évolution des prix immobiliers avec moyenne mobile, Paris, 2014T1-2023T2')
plt.xlabel('Date')
plt.ylabel('Prix M2')
plt.xticks(rotation=45)
sns.set_theme(style="whitegrid")
handles, labels = plt.gca().get_legend_handles_labels()
sorted_handles_labels = sorted(zip(handles[1:], labels[1:]), key=lambda t: t[1])  # Skip the first handle/label if it's the automatically generated one
sorted_handles, sorted_labels = zip(*sorted_handles_labels)
num_items = len(handles) - 1  
num_columns_per_row = num_items // 2 + (num_items % 2 > 0)  
plt.legend(handles=sorted_handles, labels=sorted_labels, loc='upper center',
           bbox_to_anchor=(0.5, -0.1), fancybox=True, shadow=True,
           ncol=num_columns_per_row)
plt.tight_layout()
plt.show()

Cependant, à un niveau micro, on remarque que dans certaines localités comme Paris les prix ont commencé à baisser.

## Partie II - Création des variables de contrôle

### II-1 Récupération des donnée

### Chômage

In [ ]:
#IMPORT UNEMPLOYMENT DATA
#source https://www.insee.fr/fr/statistiques/1893230
t_passage = pd.read_feather(path+"/data/interim/tble_de_passage_py.feather")[["COM","ZE","LIB_COM", "LIB_ZE", "EPCI", "LIB_EPCI"]].drop_duplicates()
unemployement = pd.read_excel(path+"/data/external/unemployment/chomage-zone-t1-2003-t3-2023.xlsx", 
                               sheet_name="txcho_ze", skiprows=5)
unemployement_long = pd.melt(unemployement, id_vars=['ZE2020', 'LIBZE2020', 'REG', 'LIBREG'],
                             var_name='Date', value_name='UNEMP')

In [ ]:
def transform_date(date):
    if date.endswith('-T1'):
        return date[:4] + '0101'
    elif date.endswith('-T2'):
        return date[:4] + '0401'
    elif date.endswith('-T3'):
        return date[:4] + '0701'
    elif date.endswith('-T4'):
        return date[:4] + '1001'
    else:
        return pd.NA

In [ ]:
unemployement_long['Date'] = unemployement_long['Date'].apply(transform_date)
unemployement_long = unemployement_long[pd.to_numeric(unemployement_long['Date'], errors='coerce') > 20131231]

In [ ]:
unemployement_long.head()

### Revenus

In [ ]:
#IMPORT AVAILABLE INCOME DATA  
#source https://www.insee.fr/fr/statistiques/3126151
#source https://www.insee.fr/fr/statistiques/6036907

income2014 = pd.read_excel(path+"/data/external/revenus/indic-struct-distrib-revenu-2014-COMMUNES/indic-struct-distrib-revenu-2014-COMMUNES/FILO_DISP_COM.xls", 
                               sheet_name="ENSEMBLE", skiprows=5)
income2014 = income2014[["CODGEO","Q214"]]

income2019 = pd.read_excel(path+"/data/external/revenus/indic-struct-distrib-revenu-2019-COMMUNES/FILO2019_DISP_COM.xlsx", 
                               sheet_name="ENSEMBLE", skiprows=5)
income2019 = income2019[["CODGEO","Q219", "GI19"]]

In [ ]:
#MERGE 
income2014_2019 = pd.merge(income2014, income2019, how='inner', on = ["CODGEO"])

In [ ]:
#ADD column Mediane_evol_diff to observe the evolution of available income between 2019 and 2019
income2014_2019["med_change"] = income2014_2019["Q219"]/income2014_2019["Q214"]
income2014_2019["CODGEO"] = income2014_2019["CODGEO"].astype(str) #preventive bug correction

In [ ]:
income2014_2019.head()

In [ ]:
sns.kdeplot(income2019[["Q219"]], fill=True, label= "2019",palette='Blues')
sns.kdeplot(income2014[["Q214"]], fill=True, label = "2014",palette='Greens')
plt.xlabel('Revenu médian par an en euros')
plt.ylabel('Densité')
plt.legend()
plt.title("Densité du revenu médian en 2014 et 2019")
plt.show()

### Densité de population

In [ ]:
#IMPORT POPULATION AND SURFACE
#https://www.insee.fr/fr/statistiques/7632565
population = pd.read_excel(path+"/data/external/pop_density/base-cc-serie-historique-2020.xlsx", 
                               sheet_name="COM_2020", skiprows=5)[["CODGEO","P20_POP","P14_POP", "SUPERF"]]
population = population[~population['CODGEO'].isin(["75056", "69123", "13055"])]  #Remove Paris, Lyon, Marseille                             
                               
population_arr = pd.read_excel(path+"/data/external/pop_density/base-cc-serie-historique-2020.xlsx", 
                               sheet_name="ARM_2020", skiprows=5)[["CODGEO","P20_POP","P14_POP", "SUPERF"]]

population_arr.loc[population_arr['CODGEO'] == "75112", 'SUPERF'] = 6.37 #correct for Bois de Vincennes
population_arr.loc[population_arr['CODGEO'] == "75116", 'SUPERF'] = 7.91 #correct for Bois de Boulogne


In [ ]:
densite = pd.concat([population, population_arr], axis=0, ignore_index=True)
#create popdensity2014 and popdensity2020 : population/surface
densite["popdensity2014"] = densite["P14_POP"] / densite["SUPERF"]
densite["popdensity2020"] = densite["P20_POP"] / densite["SUPERF"]
densite["CODGEO"] = densite["CODGEO"].astype(str) #preventive bug correction

In [ ]:
densite.head()

In [ ]:
#MERGE INCOME2014_2019 AND DENSITE
control_var = pd.merge(income2014_2019,densite,how="inner",on=["CODGEO"])

In [ ]:
plt.subplot
sns.kdeplot(densite[["popdensity2020"]], fill=True, label= "2020",palette='Blues')
plt.xlabel('Densité de population')
plt.ylabel('Densité')
plt.legend()
plt.title('Densité de la densité de population en 2014 et 2020 (tronque à 2000) ')
plt.show()

### Accessibilité aux médecins généralistes

In [ ]:
#IMPORT PHYSICISTS accessibility
#source https://data.drees.solidarites-sante.gouv.fr/explore/dataset/530_l-accessibilite-potentielle-localisee-apl/information/
physicist = pd.read_excel(path+"/data/external/physicist/Indicateur d'accessibilité potentielle localisée (APL) aux médecins généralistes.xlsx",sheet_name="APL_2019", skiprows=8)
physicist = physicist[["Code commune INSEE","APL aux médecins généralistes de moins de 65 ans"]]
#renaming variables 
physicist = physicist.rename(columns ={"Code commune INSEE": "CODGEO", "APL aux médecins généralistes de moins de 65 ans" : "Physicist_access"})
physicist = physicist.dropna()


In [ ]:
physicist.head()

In [ ]:
#MERGE
control_var = pd.merge(control_var,physicist,how="inner",on="CODGEO")

### Taux de criminalité

In [ ]:
#IMPORT CRIMINALITY
#source (not compressed version) https://www.data.gouv.fr/fr/datasets/bases-statistiques-communale-et-departementale-de-la-delinquance-enregistree-par-la-police-et-la-gendarmerie-nationales/#/resources
criminality = pd.read_csv(path+"/data/external/criminality/donnee-data.gouv-2022-geographie2023-produit-le2023-07-17.csv",sep=",")
criminality = criminality.rename(columns = {"CODGEO_2023":"CODGEO"})
criminality['tauxpourmille'] = criminality['tauxpourmille'].fillna(criminality['complementinfotaux'])
criminality19 = criminality[criminality["annee"].isin([19])]
criminality19['tauxpourmille'] = criminality19['tauxpourmille'].str.replace(",", ".").astype(float)

In [ ]:
burglary19 = criminality19[criminality19["classe"].isin(["Cambriolages de logement"])] #around 70% of Nan in the column tauxpourmille
burglary19 = burglary19[["CODGEO","tauxpourmille"]]
burglary19 = burglary19.rename(columns = {"tauxpourmille":"burglary_for_1000"})

In [ ]:
burglary19.head()

In [ ]:
assault19 = criminality19[criminality19["classe"].isin(["Coups et blessures volontaires"])] #around 60% of Nan in the column faits (number of occurence)
assault19= assault19[["CODGEO","tauxpourmille"]]
assault19 = assault19.rename(columns = {"tauxpourmille":"assault_for_1000"})

In [ ]:
assault19.head()

In [ ]:
otherassault19 = criminality19[criminality19["classe"].isin(["Autres coups et blessures volontaires"])] #around 45% of Nan in the column faits (number of occurence)
otherassault19= otherassault19[["CODGEO","tauxpourmille"]]
otherassault19 = otherassault19.rename(columns = {"tauxpourmille":"other_assault_for_1000"})

In [ ]:
otherassault19.head()

In [ ]:
destruction19 = criminality19[criminality19["classe"].isin(["Destructions et dégradations volontaires"])] #around 70% of Nan in the column faits (number of occurence)
destruction19 = destruction19[["CODGEO","tauxpourmille"]]
destruction19 = destruction19.rename(columns = {"tauxpourmille":"destruction_for_1000"})

In [ ]:
destruction19.head()

### II-2 Création d'une table variable de contrôle 

In [ ]:
#MERGE WITH control_var
control_var = pd.merge(control_var,burglary19,how="left",on="CODGEO")
control_var = pd.merge(control_var,assault19,how="left",on="CODGEO")
control_var = pd.merge(control_var,otherassault19,how="left",on="CODGEO")
control_var = pd.merge(control_var,destruction19,how="left",on="CODGEO")

In [ ]:
#MERGE WITH TABLE DE PASSAGE
control_var = pd.merge(t_passage,control_var,how="right",right_on="CODGEO", left_on="COM")

In [ ]:
control_var.head()

**Définition des variables** 

| Variable | Description  |  Source | 
|---|---|---|
| Q214   | Revenu disponible médian par unité de consommation en 2014 (€)  | INSEE  | 
| Q219   | Revenu disponible médian par unité de consommation en 2014 (€) | INSEE  | 
| GI19   | Indice de Gini en 2019  | INSEE  | 
| popdensity2014   | Densité de population en 2014 (nombre d'habitants/surface)   | INSEE  | 
| popdensity2020   | Densité de population en 2020 (nombre d'habitants/surface)  | INSEE  | 
| Physicist_access   | Indicateur d'ccessibilité potentielle localisée (APL) aux médecins généralistes* | Ministère de la santé  | 
| burglary_for_1000   | Nombre de cambriolages pour 1000 habitants en 2019  | Ministère de l'intérieur  | 
| assault_for_1000   | Nombre de coups et blessures volontaires pour 1000 habitants en 2019  | Ministère de l'intérieur  | 
| other_assault_for_1000   | Nombre d'autres coups et blessures volontaires pour 1000 habitants en 2019   | Ministère de l'intérieur  | 
| destruction_for_1000  | Nombre de destructions et dégradations volontaires pour 1000 habitants en 2019   | Ministère de l'intérieur  | 

*DREES: "L’indicateur d’accessibilité potentielle localisée (APL) a été développé pour mesurer l’adéquation spatiale entre l’offre et la demande de soins de premier recours à un échelon géographique fin. C'est un indicateur local, disponible au niveau de chaque commune, qui tient compte de l’offre et de la demande issues des communes environnantes. Calculé à l’échelle communale, l’APL met en évidence des disparités d’offre de soins qu’un indicateur usuel de densité, calculé sur des mailles beaucoup plus larges aurait tendance à masquer. Il tient également compte du niveau d’activité des professionnels en exercice ainsi que de la structure par âge de la population de chaque commune qui influence les besoins de soins."

In [ ]:
# Writing output
feather.write_feather(control_var, path+"/data/interim/TI_controls.feather")
feather.write_feather(unemployement_long, path+"/data/interim/TV_controls.feather")

### II-3 Visualisations supplémentaires des données de controle

In [ ]:
P14_POP_by_ZE = control_var.groupby('ZE')['P14_POP'].sum() #population niveau ZE en 2014
dfhealth = control_var
dfhealth["Physicist_access_pondere"] = control_var['Physicist_access'] * control_var["P14_POP"] #multiplie Physicist_access par la population
dfhealth = dfhealth.groupby('ZE')['Physicist_access_pondere'].sum() 
dfhealth = pd.merge(dfhealth,P14_POP_by_ZE,on="ZE")
dfhealth["Physicist_access_ZE"] = dfhealth["Physicist_access_pondere"]/dfhealth["P14_POP"]
dfhealth = dfhealth[["Physicist_access_ZE"]]
dfhealth["Physicist_access_ZE"] = dfhealth["Physicist_access_ZE"].astype(float)
 
full_map3 = ze_shp.merge(dfhealth, on="ZE", how="left")




plot_map(full_map3, 
         'Physicist_access_ZE', "Physicist_access en 2014 par zone d'emploi", 
         "YlGnBu")

In [ ]:
dassault = control_var
dassault["assault_for_1000_pondere"] = control_var['assault_for_1000'] * control_var["P14_POP"] #multiplie Physicist_access par la population
dassault = dassault.groupby('ZE')['assault_for_1000_pondere'].sum() 
dassault = pd.merge(dassault,P14_POP_by_ZE,on="ZE")
dassault["assault_for_1000_ZE"] = dassault["assault_for_1000_pondere"]/dassault["P14_POP"]
dassault = dassault[["assault_for_1000_ZE"]]
dassault["assault_for_1000_ZE"] = dassault["assault_for_1000_ZE"].astype(float)
 
full_map3 = ze_shp.merge(dassault, on="ZE", how="left")




plot_map(full_map3, 
         'assault_for_1000_ZE', "Aggression pour mille en France par ZE", 
         "YlGnBu")

## Partie III - Modélisation économétrique

Dans cette section consacrée à la modélisation, nous ferons appel à des méthodes économétriques afin de tenter de répondre à la problématique centrale de ce projet : dans quelle mesure la hausse récente des taux d'intérêt impacte-t-elle différemment les marchés immobiliers locaux en fonction des caractéristiques de chaque ville ?

Comme mentionné dans les sections précédentes, la disponibilité des variables d'intérêt sous une forme évolutive dans le temps est limitée, ce qui contraint l'usage de méthodes de séries temporelles en panel telles que les PVAR. Par conséquent, nous privilégions des méthodes classiques d'économétrie de panel, telles que les modèles à effets fixes.

Le modèle principal de cette analyse est un modèle à effets fixes. Il intègre comme variables d'intérêt le retard (lag) du taux d'intérêt en interaction avec différentes caractéristiques locales, tout en contrôlant les effets annuels et la structure des transactions effectuées. Des modèles supplémentaires à différents niveaux géographiques sont estimés pour tester la robustesse de nos résultats.

Les prix immobiliers utilisés sont exprimés, sauf indication contraire, en euros 2014.

In [ ]:
# Custom Functions
def merge_and_transform(base_df, irflation_df, control_df, base_col, min_transactions=30, required_count=38):
    base_df=base_df.drop(columns=["LIB_"+base_col])
    df = base_df[base_df['n_transactions'] > min_transactions]
    df['n'] = df.groupby(base_col)[base_col].transform('count')
    df = df[df['n'] == required_count].drop(columns='n')
    df = df.merge(irflation_df, on='Date', how='left')
    df = df.merge(control_df, on=base_col, how='left')
    df['prixM2'] = df['prixM2'] / df['BASE14']
    df['log_prixM2'] = np.log(df['prixM2'])
    df['Q219'] = df['Q219'] / irflation['BASE14'][20]
    df['med_change'] = (df['Q219'] / irflation_df['BASE14'][20]) - (df['Q214'] / irflation_df['BASE14'][0])
    df['dens_change'] = df['popdensity2020'] - df['popdensity2014']
    return df

In [ ]:
def group_and_calculate_diff(df, group_cols):
    df = df.sort_values(by=group_cols + ['Date'])
    grouped = df.groupby(group_cols)
    df['diff_prixM2'] = grouped['prixM2'].diff().fillna(np.nan)
    df['diff2_prixM2'] = grouped['prixM2'].diff().diff().fillna(np.nan)
    df['year'] = df['Date'].str[:4]
    df['trim'] = df['Date'].str[5:6]
    df['diff2_prop_maison'] = grouped['prop_maison'].diff().fillna(np.nan)
    df['diff_ir'] = grouped['ir'].diff().fillna(np.nan)
    df['lag_ir'] = grouped['ir'].shift(1)
    df['lag2_ir'] = grouped['ir'].shift(2)
    df['lag3_ir'] = grouped['ir'].shift(3)
    df['lag4_ir'] = grouped['ir'].shift(4)
    df['diff_lag_ir'] = grouped['lag_ir'].diff().fillna(np.nan)
    df['diff2_lag_ir'] = grouped['lag_ir'].diff().diff().fillna(np.nan)
    
    for var in ['popdensity2020', 'med_change', 'Physicist_access', 'Q219', 'prop_maison', 'assault_for_1000', 'dens_change']:
      df[f'lag3_ir_{var}'] = df['lag3_ir'] * df[var]
    df['prixM2_FE'] = grouped['prixM2'].transform(lambda x: x - x.mean()) #Within transformation
    df['log_prixM2_FE'] = grouped['log_prixM2'].transform(lambda x: x - x.mean()) #Within transformation
    df['lag3_ir_FE'] = grouped['lag3_ir'].transform(lambda x: x - x.mean()) #Within transformation
    df['prop_maison_FE'] = grouped['prop_maison'].transform(lambda x: x - x.mean()) #Within transformation
    df['popdensity2020_ir_FE'] = grouped['lag3_ir_popdensity2020'].transform(lambda x: x - x.mean()) #Within transformation
    df['med_change_ir_FE'] = grouped['lag3_ir_med_change'].transform(lambda x: x - x.mean()) #Within transformation
    df['Physicist_access_ir_FE'] = grouped['lag3_ir_Physicist_access'].transform(lambda x: x - x.mean()) #Within transformation
    df['Q219_ir_FE'] = grouped['lag3_ir_Q219'].transform(lambda x: x - x.mean()) #Within transformation
    df['prop_maison_ir_FE'] = grouped['lag3_ir_prop_maison'].transform(lambda x: x - x.mean()) #Within transformation
    df['assault_for_1000_ir_FE'] = grouped['lag3_ir_assault_for_1000'].transform(lambda x: x - x.mean()) #Within transformation
    df['dens_change_ir_FE'] = grouped['lag3_ir_dens_change'].transform(lambda x: x - x.mean()) #Within transformation
    return df.reset_index(drop=True)

In [ ]:
def run_regression(df, formula):
    model = smf.ols(formula, data=df)
    results = model.fit()
    return(results)

In [ ]:
def aggregate_data(df, group_cols, agg_dict):
    agg_df = df.groupby(group_cols).agg(agg_dict).reset_index()
    agg_df['popdensity2020'] = agg_df['P20_POP'] / agg_df['SUPERF']
    agg_df['popdensity2014'] = agg_df['P14_POP'] / agg_df['SUPERF']
    return agg_df.drop(columns=['P20_POP', 'P14_POP', 'SUPERF'])

In [ ]:
def corplot(df, nom):
  plt.figure(figsize=(10, 8))
  sns.heatmap(df, annot=True, cmap='coolwarm')
  plt.title('Correlations intra-ville pour '+nom)
  plt.show()

In [ ]:
def cor_ville(df, nom):
  df = df[df['LIB_COM']==nom]
  df = df[["prixM2", "ir","lag_ir","lag2_ir","lag3_ir","lag4_ir", "prop_maison"]].dropna().corr()
  return(df)

In [ ]:
# Reading Data
base_path = path + "/data/interim/"
immo_epci = pd.read_feather(base_path + "immo_panel_epci_py.feather")
immo_ze = pd.read_feather(base_path + "immo_panel_ze_py.feather")
immo_com = pd.read_feather(base_path + "immo_panel_com_py.feather")
immo_full = pd.read_feather(base_path + "immo_panel_full_py.feather")
unemployment = pd.read_feather(base_path + "TV_controls.feather")
control = pd.read_feather(base_path + "TI_controls.feather")
irflation = pd.read_feather(base_path + "irflation.feather")

In [ ]:
# Aggregate data
agg_dict_control = {
    'Q219': 'mean', 'Q214': 'mean',
    'P20_POP': 'sum', 'P14_POP': 'sum',
    'SUPERF': 'sum', 'Physicist_access': 'mean',
    'assault_for_1000': 'mean'
}
control_ze = aggregate_data(control, ['ZE', 'LIB_ZE'], agg_dict_control)
control_epci = aggregate_data(control, ['EPCI', 'LIB_EPCI'], agg_dict_control)

In [ ]:
control_ze

In [ ]:
# Processing for immo_reg_com
immo_reg_com = merge_and_transform(immo_com, irflation, control, 'COM')
immo_reg_com_diff = group_and_calculate_diff(immo_reg_com, ['COM'])

Pour obtenir un panel équilibré avec des prix au m2 suffisament stable nous gardons uniquement les villes, les ZE et les EPCI qui ont eu au moins 30 transactions par trimestre. Cela a un impact sur la distribution des prix dans notre sample, en ce qui concerne les villes, les deux densités sont représentées ci-dessous.

In [ ]:
filtered_data = immo_com[immo_com['n_transactions'] > 30][['COM', 'Date', 'prixM2', 'prop_maison', 'n_transactions']]
merged_data = filtered_data.merge(irflation, on='Date', how='left').merge(control, on='COM', how='left')
grouped = merged_data.groupby('COM').agg(
    n=('n_transactions', lambda x: (x > 30).sum()),
    popdensity2020=('popdensity2020', 'mean'),
    prixM2=('prixM2', 'mean')
).reset_index()
grouped['Sampling'] = grouped['n'].apply(lambda x: 'In sample' if x == 38 else 'Out of sample')
plt.figure(figsize=(10, 6))
ax=sns.kdeplot(data=grouped, x='prixM2', hue='Sampling', common_norm=False)
plt.title('Density Plot of PrixM2')
plt.xlabel('PrixM2')
plt.ylabel('Density')
ax.set_xlim(-1000, 25000)
plt.show()

L'effet est encore plus marqué pour les densités. 

In [ ]:
plt.figure(figsize=(10, 6))
ax=sns.kdeplot(data=grouped, x='popdensity2020', hue='Sampling', common_norm=False)
plt.title('Density Plot of PrixM2')
plt.xlabel('Densité de population en 2020')
plt.ylabel('Density')
ax.set_xlim(-1000, 25000)
plt.show()

Ainsi, il faut considérer que nos résultats ne sont pas représentatifs de l'ensemble des marchés immobiliers français mais uniquement des marchés immobiliers assez actifs.

In [ ]:
# Processing for immo_reg_ze
immo_reg_ze = merge_and_transform(immo_ze, irflation, control_ze, 'ZE')
immo_reg_ze_diff = group_and_calculate_diff(immo_reg_ze, ['ZE', 'LIB_ZE'])
immo_reg_ze_diff = immo_reg_ze_diff.merge(unemployment[['ZE2020', 'Date', 'UNEMP']], left_on=['ZE', 'Date'], right_on=['ZE2020', 'Date'], how='left')
immo_reg_ze_diff['lag3_UNEMP'] = immo_reg_ze_diff.groupby(['ZE', 'LIB_ZE'])['UNEMP'].shift(3)
immo_reg_ze_diff['lag3_UNEMP_ir'] = immo_reg_ze_diff['lag3_UNEMP']*immo_reg_ze_diff['lag3_ir'] 
grouped = immo_reg_ze_diff.groupby(['ZE', 'LIB_ZE'])
immo_reg_ze_diff['lag3_UNEMP_FE'] = grouped['lag3_UNEMP'].transform(lambda x: x - x.mean()) #Within transformation
immo_reg_ze_diff['lag3_UNEMP_ir_FE'] = grouped['lag3_UNEMP_ir'].transform(lambda x: x - x.mean()) #Within transformation

In [ ]:
# Processing for immo_reg_epci
immo_reg_epci = merge_and_transform(immo_epci, irflation, control_epci, 'EPCI')
immo_reg_epci_diff = group_and_calculate_diff(immo_reg_epci, ['EPCI', 'LIB_EPCI'])

### Test de Granger

Le marché immobilier, étant moins liquide que les marchés financiers, ne révèle l'impact du taux d'intérêt sur les prix qu'après un certain délai. Il est courant dans la littérature de considérer le retard du taux d'intérêt plutôt que le taux lui-même, comme illustré par Harris (1989) dans _The Effect of Real Rates of Interest on Housing Prices_. Cependant, cette transmission varie selon les caractéristiques des marchés immobiliers et le contexte macroéconomique. Ainsi, pour déterminer le nombre approprié de retards du taux d'intérêt dans l'analyse des marchés locaux en France, nous employons un test de Granger. Ce test évalue si une série temporelle améliore la prédiction d'une autre série temporelle et, par conséquent, si elle est pertinente pour son analyse. $$\Delta ({p}/m^{2})_t \sim \Delta L^n(i_t)$$ Nous cherchons à déterminer si l'utilisation du taux d'intérêt avec plusieurs retards apporte des informations sur les prix immobiliers. L'hypothèse nulle est l'absence de pouvoir prédictif. Pour obtenir des séries stationnaires, nous utilisons la différence. Bien qu'il existe des méthodes pour généraliser le test de Granger à des données en panel (Holtz-Eakin et al., 1988), celles-ci n'ont pas été développées en Python. Pour obtenir une idée générale du nombre de retards requis, nous appliquons le test à quelques villes de tailles variées choisies aléatoirement.

In [ ]:
# Granger Causality Test
for i in ["Paris 14","Clermont-Ferrand","Le Grau-du-Roi", "Cagnes-sur-Mer", "Savigny-sur-Orge"]:
  print(i)
  granger_test3 = grangercausalitytests(immo_reg_com_diff[immo_reg_com_diff["LIB_COM"]==i][['diff_prixM2', 'diff_ir']].dropna(), maxlag=5, verbose=True)

Il apparaît que, selon la commune, si un effet existe, entre 1 et 3 trimestres sont nécessaires pour que le taux d'intérêt ait un pouvoir explicatif sur les prix immobiliers.

### Corrélations

Afin d'observer les interactions entre les variables d'intérêt choisies et d'obtenir une première impression de ce que la modélisation peut apporter, il est utile de construire des matrices de corrélations. D'abord, une matrice pour l'ensemble du panel, puis pour les trois premières villes utilisées lors du test de Granger.

In [ ]:
# Total 
immo_reg_corr =immo_reg_com_diff[["prixM2", "ir","lag_ir","lag2_ir","lag4_ir","popdensity2020", "med_change", "Physicist_access", "Q219", "prop_maison", "assault_for_1000", "dens_change"]].dropna().corr()
corplot(immo_reg_corr, "Total")

Cette matrice révèle une corrélation relativement faible entre les prix et le taux d'intérêt, quel que soit le nombre de retards. En revanche, les corrélations sont significatives entre les prix et la densité en 2020, l'évolution des revenus médians dans la commune, la proportion de maisons dans le total des transactions, et les changements de densité. La faible corrélation entre le taux d'intérêt et les prix s'explique probablement par la dimension NN de notre panel, bien plus importante que la dimension TT, le taux d'intérêt étant identique pour toutes les villes à chaque période.

Pour vérifier cela, il est possible d'établir des matrices de corrélations pour quelques villes, en excluant les variables constantes dans le temps. Par exemple, pour le 14e arrondissement de Paris : 

In [ ]:
# Intra villes
for i in ["Paris 14","Clermont-Ferrand","Le Grau-du-Roi"]:
  corplot(cor_ville(immo_reg_com_diff, i), i)

Les corrélations avec le taux d'intérêt sont plus fortes, toujours négatives, et généralement plus élevées dans les deux villes de taille supérieur. De plus, la corrélation est plus forte au premier retard à Paris, tandis que pour Clermont-Ferrand et le Grau-du-Roi, elle est plus importante au troisième retard. Cela pourrait indiquer un effet différencié du taux d'intérêt selon la taille de la ville.


### Modèles communaux

Dans le contexte de données de panel, différents problèmes se posent à l'estimateur OLS (MCO en français), souvent appelé Pooled-OLS (cf. On The Pooling Of Time Series And Cross Section Data, Yair Mundlak, 1978). Deux méthodes fréquemment utilisées pour pallier les limites de POLS sont l'estimation avec effets fixes (fixed-effects) ou effets aléatoires (random-effects). Un modèle à effets fixes est privilégié dans ce projet car il permet d'appréhender les variations intra-groupe tout en contrôlant l'hétérogénéité inobservable constante dans le temps (par exemple, la présence d'une rivière dans la ville). Ce choix est également courant dans la littérature sur l'évaluation des biens immobiliers. Pour plus d'informations sur la méthodologie générale appliquée aux données de panel, voir Econometric Analysis of Cross Section and Panel Data, Wooldridge, 2001.

#### Modèle Introductif - Modèle simple du taux d'intérêt

Pour introduire le modèle principal une régression simple est réalisée via un pooled-OLS sans effets croisés (l'utilisation d'effets fixes est pour l'instant impossible car cela controlerait pour toute l'hétérogéinité innobservable constante dans le temps par individu et certaines de nos variables explicatives sont constantes dans le temps) : $$p_{nt}= \beta_0+\beta_iL^3(i_t)+\beta_MM_{nt} + \beta_c X_{nc}+\beta_{D}D+\varepsilon_t$$ avec $p_{nt}$ le prix des appartements et maisons dans ville $n$ à la période t, $L^3(i_t)$ le troisième retard du taux d'interet à la période t, $M_{nt}$ la proportion de maisons dans le total des transactions immobilières à la période t pour la ville n, $X_{nc}$ la matrice des variables des contrôles constante dans le temps pour la ville $n$ (densité de population en km2 en 2019, revenus médian en 2019, accès à un médecin généraliste, taux d'agressions pour 1000, et la différence de densité et de revenus entre respectivement, 2020/2014 et 2019/2014). $D$ est une variable muette servant à controler les effets fixes de chaque trimestre (par exemple, la situation macroéconomique nationale ou les confinements pendant la période covid). 

Ainsi, cette régression a pour objectif de vérifier la cohérence des résultats et d'identifier certains problèmes de données (par exemple des valeurs manquantes au niveau des arrondissements de Paris, Lyon ou Marseille). 

In [ ]:
# Model 1a
model1a= 'prixM2 ~ lag3_ir + (popdensity2020 + med_change + Physicist_access + Q219 + prop_maison + assault_for_1000 + dens_change) + prop_maison + C(Date)'
mod1a = run_regression(immo_reg_com_diff, model1a)
test_breuschpagan = het_breuschpagan(mod1a.resid,  mod1a.model.exog)
test_breuschpagan

Ce modèle ne semble pas souffrir d'hétéroscédasticité (pval=0.5>0.05)

In [ ]:
mod1a.summary()

Les résultats confirment l'influence positive de la densité de population et du revenu médian communal sur les prix immobiliers. Les maisons ont généralement un prix au mètre carré inférieur aux appartements, d'où un effet négatif pour la variable maison. Le taux d'intérêt a un impact significatif d'environ -788€ par m² pour un point de taux supplémentaire. La seule surprise vient de la criminalité, qui a un effet positif, capturant peut-être un effet de mobilité de population plus élevé (agressions plus nombreuses dans zones avec plus de passage?).

#### Modèle 1 - Modèle complet avec effets fixes

Le modèle principal de cette analyse, similaire au modèle introductif, se distingue par l'inclusion de coefficients d'interaction entre le taux d'intérêt et les autres variables. Cela permet d'utiliser un modèle à effets fixes, les variables constantes dans le temps devenant dynamiques grâce à l'interaction avec le taux d'intérêt. Le modèle, après application d'une transformation within, devient : $$p_{nt}-\bar{p_{n}}= \beta_0+\beta_i[L^3(i_t)-\bar{L^3(i_t)}]+\beta_M[M_{nt}-\bar{M_{n}}] + \beta_{ic}[L^3(i_t)X_{nc}-\bar{L^3(i)X_{nc}}]+\beta_{im}[L^3(i_t)M_{nt}-\bar{L^3(i)M_{n}}]+\beta^{FE}_{D}D+\varepsilon_t$$ $$\iff \ddot{p_{nt}}= \beta^{FE}_0+\beta^{FE}_i\ddot{L^3(i_t)}+\beta^{FE}_M\ddot{M_{nt}} + \beta^{FE}_{ic}[\ddot{L^3(i_t)X_{nc}}]+\beta^{FE}_{im}[\ddot{L^3(i_t)M_{nt}}]+\beta^{FE}_{D}D+\varepsilon_t$$ Remarque: Inclure une variable muette permet directement d'obtenir l'estimation à effets-fixe (c'est ce qui a été utilisé pour les effets fixe trimestriels) cependant cette approche présente un coût calculatoire élevé avec beaucoup de groupes, la transformation within y est donc souvent préféré (où pour chaque variable est réduite de la moyenne du groupe, ici la ville)

Remarque: La transformation within peut si deux variables d'interactions changent au cours du temps et sous certaines conditions provoquer un biais (voir _Interactions in Fixed Effects Regression Models_, Giesselmann et Schmidt-Catran, 2018). Ce n'est pas le cas pour la plupart de nos coefficients, sauf $\beta_{im}$ qui représente l'intéraction entre le taux d'interet et la proportion de maison dans le total des transactions immobilières

In [ ]:
# Model 1b
model1b_formula = 'prixM2_FE ~ lag3_ir_FE+prop_maison_FE + (popdensity2020_ir_FE + med_change_ir_FE + Physicist_access_ir_FE + Q219_ir_FE + assault_for_1000_ir_FE + dens_change_ir_FE + prop_maison_ir_FE) + C(Date)'
mod1b = run_regression(immo_reg_com_diff, model1b_formula)

test_breuschpagan = het_breuschpagan(mod1b.resid,  mod1b.model.exog)
test_breuschpagan

Ce modèle ne semble pas souffrir d'hétéroscédasticité (pval=0.39>0.05)

In [ ]:
mod1b.summary()

Ce modèle suggère qu'une hausse du taux d'intérêt a un effet plus marqué dans les villes densément peuplées, avec une baisse de 0.0248€ des prix au m2 par point de taux d'interet en plus par personne par km2. Pour Paris cet effet serait d'environ 500€ par point de taux d'interet. Concernant les revenus médians, on observe une baisse de 0.0414€ par point d'intérêt supplémentaire pour chaque euros de revenu médian en 2019, soit environ 1200€ de baisse pour Paris. Un effet similaire est observé dans des villes en croissance (économique et démographique). Les villes avec un bon niveau d'accès au soins serait également plus touché comme les villes avec plus d'agressions (variable qui se comporte peut-être comme un proxy des zones métropolitaines), cependant ces deux variables ont une significativité pratique assez faible (l'échelle sur laquelle ces variables sont distribuées est assez petite). Le $R^2$ affiché est "within" c'est à dire qu'il représente la partie de la variance expliquée à l'intérieur du groupe, dans le cas présent notre modèle explique environ 25% de la variance des prix immobilier dans une même ville dans la dimension temporelle. 

Ces résultats semblent cohérents avec l'hypothèse d'effets spéculatifs et/ou de substitutions. Pour vérifier la robustesse de ces résultats plusieurs spécifications supplémentaires sont dévelopées. 

In [ ]:
residuals = mod1b.resid
plt.figure(figsize=(8, 6))
sns.kdeplot(residuals, bw_adjust=7, shade=True)  # Adjust bandwidth as needed
plt.title('Distribution des résidus')
plt.xlim(-5000, 5000)
plt.xlabel('Residuals')
plt.ylabel('Density')
plt.show()

Les résidus semblent être distribué par une Gaussienne centrée sur 0. 

#### Modèle 1c - Modèle log-niveau

Une critique possible de notre modèle est l'existance d'effets d'échelles relativement fort sur les prix, les prix immobilier pouvant varier fortement d'une ville à une autre. Ce modèle utilise donc une spécification log-niveau.
$$ \ddot{log(p_{nt})}= \beta^{FE}_0+\beta^{FE}_i\ddot{L^3(i_t)}+\beta^{FE}_M\ddot{M_{nt}} + \beta^{FE}_{ic}[\ddot{L^3(i_t)X_{nc}}]+\beta^{FE}_{D}D+\varepsilon_t$$

In [ ]:
# Model 1c
model1c_formula = 'log_prixM2_FE ~ lag3_ir_FE+prop_maison_FE + (popdensity2020_ir_FE + med_change_ir_FE + Physicist_access_ir_FE + Q219_ir_FE + assault_for_1000_ir_FE + dens_change_ir_FE + prop_maison_ir_FE) + C(Date)'
mod1c = run_regression(immo_reg_com_diff, model1c_formula)

test_breuschpagan = het_breuschpagan(mod1c.resid,  mod1c.model.exog)
test_breuschpagan

Ce modèle ne semble pas souffrir d'hétéroscédasticité (pval=0.17>0.05)

In [ ]:
mod1c.summary()

Les résultats sont cohérents avec le premier modèle, avec une augmentation significative du $R^2$, indiquant que le modèle explique 60% de la variance du log des prix dans une même commune.

#### Modèle 2 - Modèle avec prix dé-tendanciés

Ce modèle s'attaque au risque de corrélations fallacieuses dû au caractère non stationnaire des prix immobiliers. Il consiste en une estimation en deux étapes, sans inclure d'effets fixes pour simplifier les calculs.
1) Régression des prix sur le temps avec t le nombre de trimestres depuis le début du panel pour chaque groupe : $$ p_{nt}=\beta_0+\beta_{tn}t$$ 

2) Régression des prix réduit de la tendance linéaire estimé sur le taux d'interet et les interactions : $$(p_{nt}-\beta_{nt}\times t)= \beta_0+\beta_iL^3(i_t)+\beta_MM_{nt} + \beta_c X_{nc}+\beta_{ic}L^3(i_t)X_{nc}+\beta_{iM}L^3(i_t)M_{nt}+\beta_{D}D+\varepsilon_t $$

Remarque : enlever la tendance de l'évolution des prix ne permet pas de rendre ce processus stationaire mais réduit tout de même le risque d'obtenir des résultats liée à la hausse des prix sur la période

In [ ]:
#DETRENDED REGRESSION
immo_reg_com_diff['time'] = immo_reg_com_diff.groupby('COM').cumcount() + 1
df_reg = immo_reg_com_diff.groupby('COM').apply(lambda x: smf.ols('prixM2 ~ time', data=x).fit().resid + x['prixM2']).reset_index(name='detrended_variable')
reg_immo2 = immo_reg_com_diff.groupby('COM').apply(lambda x: smf.ols('prixM2 ~ time', data=x).fit().params['time']).reset_index(name='estimate')
vroom = immo_reg_com_diff.reset_index(drop=False).merge(reg_immo2, on='COM')
vroom['prixM2_dt'] = vroom['prixM2'] - vroom['time'] * vroom['estimate']

model_detrend= 'prixM2_dt ~ lag_ir * (popdensity2020 + med_change + Physicist_access + Q219 + prop_maison + assault_for_1000 + dens_change) + prop_maison + C(AN)'
mod2 = run_regression(vroom, model_detrend)

test_breuschpagan = het_breuschpagan(mod2.resid,  mod2.model.exog)
test_breuschpagan

Ce modèle souffre d'hétéroscédasticité (p-value=1.08e-17), les résultats sont donc affichés sous forme robuste:

In [ ]:
mod2.get_robustcov_results(cov_type='HC2').summary()

Ce modèle à la particularité d'avoir peu de coefficients statistiquement signifactifs mais les effets croisés taux d'interets densité reste significatifs à un niveau inférieur à 1% et l'interactions avec les revenus est significative à un niveau d'environ 6%. De plus, le coefficient associé au chômage semble capter la majorité de l'effet précédemment attribué aux revenus, suggérant que les revenus médians pourraient être un proxy de la santé économique locale.

#### Visualisation des prix sans tendance linéaire

In [ ]:
plot_data = vroom[vroom['LIB_COM'].str.startswith('Paris 14')].copy()
plot_data['prixM2_noncorrige'] = plot_data['prixM2'] * plot_data['BASE14']
plot_data['Date'] = pd.to_datetime(plot_data['Date'], format='%Y%m%d')
plot_data_melted = plot_data.melt(id_vars='Date', value_vars=['prixM2', 'prixM2_dt', 'prixM2_noncorrige'])

plt.figure(figsize=(10, 6))
sns.lineplot(data=plot_data_melted, x='Date', y='value', hue='variable')
plt.title("Évolution des prix, Paris 14e, nominal, euros 2014 et euros 2014 detrend")
plt.xlabel("Date")
plt.ylabel("Value")
plt.legend(title='Variable')
plt.show()

En vert sont représenté les prix non corrigés de l'inflation (jamais utilisé dans le cadre de cette modélisation), en bleu les prix en euros 2014 et en orange les prix sans tendance linéaire.

### Modèle ZE (Zone d'Emploi)

#### Modèle 3 - Spécification ZE du modèle complet

Dans ce modèle, la zone d'emploi remplace la ville comme unité d'analyse, avec l'ajout du taux de chômage au même retard que le taux d'interet pour simplifier l'interprétation.

$$\ddot{p_{nt}}= \beta^{FE}_0+\beta^{FE}_i\ddot{L^3(i_t)}+\beta^{FE}_V\ddot{Z_{nt}} + \beta^{FE}_{IC}[\ddot{L^3(i_t)X_{nc}}]+\beta^{FE}_{IV}[\ddot{L^3(i_t)Z_{nt}}]+\beta^{FE}_{D}D+\varepsilon_t$$
 $Z_{nt}$ inclus toutes les charactéristiques variante dans le temps par zone d'emploi (ie. chomage et proportion de maisons vendues dans le total). 

In [ ]:
# Part 4: Linear Regression Models
# Model mod3
mod3_formula = 'prixM2_FE ~ lag3_ir_FE+prop_maison_FE+lag3_UNEMP_FE+lag3_UNEMP_ir_FE + (popdensity2020_ir_FE + med_change_ir_FE + Physicist_access_ir_FE + Q219_ir_FE + assault_for_1000_ir_FE + dens_change_ir_FE + prop_maison_ir_FE) + C(Date)'
mod3 = run_regression(immo_reg_ze_diff, mod3_formula)

test_breuschpagan = het_breuschpagan(mod3.resid,  mod3.model.exog)
test_breuschpagan

Ce modèle souffre également d'hétéroscédasticité (p-value=8.73e-91), les résultats sont donc affichés sous forme robuste:

In [ ]:
mod3.get_robustcov_results(cov_type='HC2').summary()

Les résultats sont cohérents avec nos résultats précédents mais on remarque que le coefficients associés au chomage capture la majorité de l'effet précédement attribué aux revenus. Il semblerait que les revenus médians soit un proxy général de la santé économique locale.

### Modèles EPCI (Établissement Public de Coopération Intercommunale)

#### Modèle 4 - Spécification EPCI du modèle complet

$$\ddot{p_{nt}}= \beta^{FE}_0+\beta^{FE}_i\ddot{L^3(i_t)}+\beta^{FE}_M\ddot{M_{nt}} + \beta^{FE}_{IC}[\ddot{L^3(i_t)X_{nc}}]+\beta^{FE}_{IM}[\ddot{L^3(i_t)M_{nt}}]+\beta^{FE}_{D}D+\varepsilon_t$$

In [ ]:
# Model mod5
mod5_formula = 'prixM2_FE ~ lag3_ir_FE+prop_maison_FE + (popdensity2020_ir_FE + med_change_ir_FE + Physicist_access_ir_FE + Q219_ir_FE + assault_for_1000_ir_FE + dens_change_ir_FE + prop_maison_ir_FE) + C(Date)'
mod5 = run_regression(immo_reg_epci_diff, mod5_formula)

test_breuschpagan = het_breuschpagan(mod5.resid,  mod5.model.exog)
test_breuschpagan

Le modèle souffre d'hétéroscédasticité (p-value=6.27e-43), les résultats sont donc affichés sous forme robuste:

In [ ]:
mod5.get_robustcov_results(cov_type='HC2').summary()

L'estimation du modèle au niveau EPCI révèle des résultats similaires, bien que l'effet total du taux d'intérêt soit réduit, à cause du coefficient associé à $E[B^{FE}_i|(\ddot{L^3(i_t)X_{nc}})=0]$  qui est positif et est égal à 1882€ même si celui-ci n'est pas significatif à 5%. Par ailleurs, l'impact de l'évolution des revenus est plus prononcé que dans les autres modèles.